In [ ]:
# import libraries
# python 3.12.7
import pdfplumber
import pandas as pd 
import re
import os
from datetime import datetime
import plotly.express as px
from pandas.tseries.offsets import YearEnd, QuarterEnd

## Variables

In [ ]:
date_pattern = re.compile(r'[0-9]{2}/{1}[0-9]{2}/{1}[0-9]{2}')
item_number = re.compile(r' \([0-9]{6,6}\)')
cheque_number = re.compile(r' F[0-9]{6,6} ')
strata_corp = 'Vbcs'
vbcs_table = pd.DataFrame()
#pdf_path = "data/2024Q4.pdf"  # Path to your PDF file



In [ ]:
# initial process pdfs
pdf_path = "data/"  # Path to your PDF file
pdf_files = [file for file in os.listdir('data') if file.endswith('.pdf')]
concat_list = pd.Series([])

for file in pdf_files:
    file_path =  pdf_path + file
    pdf = pdfplumber.open(file_path)
    number_pages = len(pdf.pages)
    
    for pages in range(0,number_pages): 
        page = pdf.pages[pages]
        text = page.extract_text()
        separator = "Payee Cheque No Exp Date Details Amount($)\n"
        try:
            table = text.split(separator)[1]
            table = table.split("\n")
            table = pd.Series(table)
            eligible_rows = table.str.contains(item_number)
            table = table[eligible_rows]
            concat_list =  pd.concat([concat_list,table])
        except:
            print("No more pages")
            break


## Functions

In [ ]:

def process_payee(Line_Item, Item_Number, Date_Pattern, Cheque_Number):
   
   """
    line items where the strata is NOT the cost center
   """

   cost = re.split(Item_Number, Line_Item)[1]  # print whatever comes after the 6 numbers
   cost_date = re.search(Date_Pattern, Line_Item).group()
   payee = re.split(Cheque_Number, Line_Item)[0]
   details = re.split(Cheque_Number, Line_Item)[1]
   details = re.split(Item_Number, details)[0]
   details = re.split(Date_Pattern, details)[1]

   new_row = pd.DataFrame([{'Payee': payee,
    'Details': details,
    'Cost date': cost_date,
    'Cost': cost}])
   
   return(new_row, payee)

def process_item(Line_Item, Item_Number, Date_Pattern, Payee):
    cost_date = re.search(Date_Pattern, Line_Item).group()
    cost = re.split(Item_Number, Line_Item)[1]
    details = re.split(Item_Number, Line_Item)[0]
    details = re.split(Date_Pattern, details)[1]

    new_row = pd.DataFrame([{'Payee': Payee,
        'Details': details,
        'Cost date': cost_date,
        'Cost': cost}])
    
    return(new_row)

In [ ]:
vbcs_table = pd.DataFrame()

for line_item in concat_list:
    if len(re.split(cheque_number, line_item)) == 2:
        new_row, payee = process_payee(line_item, item_number, date_pattern, cheque_number)
        vbcs_table = pd.concat([vbcs_table, new_row])

    elif len(re.split(cheque_number, line_item)) == 1:
        new_row = process_item(line_item, item_number, date_pattern, payee)
        vbcs_table = pd.concat([vbcs_table, new_row])

    else:
        print("Unaccounted for item: "+line_item+"\n")    

print(vbcs_table)


In [ ]:
# convert columns to types
vbcs_table['Cost date'] = pd.to_datetime(vbcs_table['Cost date'], format='%d/%m/%y')
vbcs_table['Cost'] = vbcs_table['Cost'].str.replace(',', '').astype(float)
print("Total cost = " + str(round(vbcs_table['Cost'].sum(),2)))
vbcs_table



## Plots

### Previous four quarters

In [ ]:
last_quarter_end = pd.Timestamp('now') - QuarterEnd(1)
last_quarter_start_previous_year = last_quarter_end - pd.DateOffset(years = 1) + pd.DateOffset(days = 1)# - QuarterEnd(1)#+ pd.DateOffset(days = 1)

prevous_four_quarters = vbcs_table[
    (vbcs_table['Cost date'] >= last_quarter_start_previous_year) &
    (vbcs_table['Cost date'] <= last_quarter_end)
]

In [ ]:
toplot = prevous_four_quarters
# Group by Payee and sum Cost
payee_totals = toplot.groupby('Payee', as_index=False)['Cost'].sum()

max_payee = max(payee_totals['Cost']) # used below for formatting

fig = px.bar(
    payee_totals,
    x='Payee',
    y='Cost',
    title='Total Cost by Payee',
    subtitle='Previous four quarters',
    labels={'Cost': 'Total Cost ($)'},
    color='Payee',
    text='Cost'
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_layout(yaxis_range=[0, max_payee+max_payee/10]) # to ensure it's not cutoff
fig.show()

### Previous calendar year

In [ ]:
last_calendar_year = (pd.Timestamp('now') - pd.DateOffset(years = 1)).year
previous_calendar_year = vbcs_table[vbcs_table['Cost date'].dt.year == last_calendar_year]

In [ ]:
toplot = previous_calendar_year
#toplot = toplot[toplot['Payee'] != 'Gio']

# Group by Payee and sum Cost
payee_totals = toplot.groupby('Payee', as_index=False)['Cost'].sum()

max_payee = max(payee_totals['Cost']) # used below for formatting

fig = px.bar(
    payee_totals,
    x='Payee',
    y='Cost',
    title='Total Cost by Payee',
    subtitle='Previous calendar year',
    labels={'Cost': 'Total Cost ($)'},
    color='Payee',
    text='Cost'
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_layout(yaxis_range=[0, max_payee+max_payee/10]) # to ensure it's not cutoff
fig.show()